In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [2]:
data = pd.read_csv(r"C:\Users\Seif Elmoushy\Downloads\archive (12)\Dataset_summarized11_1label-16.csv", encoding='ISO-8859-1')


In [3]:
# 1
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=16)
model = model.to('cuda')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'pre_classi

In [15]:
model_save_path = 'Dataset_summarized11_1label-16_25-ep_andaccuracy96%.pt'
model.load_state_dict(torch.load(model_save_path))

<All keys matched successfully>

In [16]:
# Split data into train and validation sets
X = list(data["post"].astype(str))
y = list(data["type"].astype(int))
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)

# Tokenize data and create torch datasets
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

C:\Users\Seif Elmoushy\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_split.py:1608: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return floored.astype(np.int)
C:\Users\Seif Elmoushy\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_split.py:1608: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify

In [17]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [18]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [19]:
def compute_metrics(pred):
    labels = pred.label_ids
    pred = np.argmax(pred.predictions, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='macro')
    precision = precision_score(y_true=labels, y_pred=pred, average='macro')
    f1 = f1_score(y_true=labels, y_pred=pred, average='macro')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}



In [20]:
args = TrainingArguments(
    output_dir="output",
    #num_train_epochs=10,
    per_device_train_batch_size=4
)

In [21]:
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

In [22]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=None,
    compute_metrics=compute_metrics
)

In [20]:
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

In [21]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=None,
    compute_metrics=compute_metrics
)

In [ ]:
#trainer.train()

In [23]:
result = trainer.evaluate()
print(result)

{'eval_loss': 0.24481311440467834, 'eval_accuracy': 0.9602534562211982, 'eval_precision': 0.9455041475898489, 'eval_recall': 0.9361163416024074, 'eval_f1': 0.9389387587663164, 'eval_runtime': 49.2816, 'eval_samples_per_second': 35.226, 'eval_steps_per_second': 4.403}
